In [ ]:
!pip install torch -Uq

In [ ]:
!pip install transformers -Uq

In [ ]:
!pip install tiktoken -Uq

In [ ]:
!pip install datasets -Uq

In [ ]:
!pip install matplotlib -Uq

In [ ]:
import json

class Tokenizer:
  def __init__(self, vocab_path):
    with open(vocab_path, "r", encoding="utf-8") as f:
      self.vocab = json.load(f)
      self.reverse_vocab = {v: k for k, v in self.vocab.items()}

  def encode(self, text):
    tokens = []

    for word in text.split():
      i = 0
      while i < len(word):
        found_match = False
        for j in range(len(word), i, -1):
          subword = word[i:j]
          if subword in self.vocab:
            tokens.append(self.vocab[subword])
            i = j
            found_match = True
            break
        if not found_match:
          tokens.append(self.vocab.get("<unk>", 0))
          i += 1
      tokens.append(self.vocab.get(" ", 1))
    if tokens:
      tokens.pop()
    return tokens

  def tokenize(self, text):
    token_ids = self.encode(text)
    return [self.reverse_vocab[id] for id in token_ids]

  def decode(self, ids):
    text = ""
    for id in ids:
      text += self.reverse_vocab.get(id, "<unk>")
    return text

In [ ]:
tokenizer = Tokenizer("tokenizer_1.json")

In [ ]:
import plotly.graph_objects as go
import plotly.offline

In [ ]:
def plot_dots(sentences_data, title, dims=[0, 1, 2]):
  data = [
    go.Scatter3d(
      x=sentence_data["words"][:, dims[0]],
      y=sentence_data["words"][:, dims[1]],
      z=sentence_data["words"][:, dims[2]],
      mode="markers+text",
      marker=dict(
        size=6,
        color=sentence_data["color"],
      ),
      text=sentence_data["labels"],
      hoverinfo="text",
    ) for sentence_data in sentences_data
  ]

  layout = go.Layout(
    scene=dict(
      xaxis_title="Sertlik",
      yaxis_title="Parlaklık",
      zaxis_title="Kırmızılık",
    ),
    title=title,
  )

  fig = go.Figure(data=data, layout=layout)
  plotly.offline.iplot(fig)

In [ ]:
data = "the capital of the united states"
target = "capital of the united stetes is"

In [ ]:
ids = tokenizer.encode(data)

len(ids), ids

(12, [0, 61, 1, 61, 2, 61, 0, 61, 3, 61, 4, 58])

In [ ]:
ids_t = tokenizer.encode(target)

len(ids_t), ids_t

(16, [1, 61, 2, 61, 0, 61, 3, 61, 58, 62, 62, 62, 62, 58, 61, 5])

Context length, bir dil modelinin aynı anda işleyebildiği en uzun giriş + çıkış token toplamıdır.

In [ ]:
context_length = 32 #Gpt-4o 128k

Eğer Context length dolmazsa sonu "<<c>ped>" ile doldurulur

##Basit Positional Embeding

In [ ]:
import torch

In [ ]:
sentence = "the capital of united states and the capital of france"

In [ ]:
torch.manual_seed(42) #embedding değerleri her seferinde aynı olacak

In [ ]:
embeddings = torch.nn.Embedding(num_embeddings=64, embedding_dim=4)

In [ ]:
embeddings

In [ ]:
pos_embeddings = torch.nn.Embedding(num_embeddings=context_length, embedding_dim=4) #konum için dimlik değerler

In [ ]:
pos_embeddings

In [ ]:
tokens = tokenizer.encode(sentence)

In [ ]:
tokens = torch.tensor(tokens)

In [ ]:
tokens

In [ ]:
meanings = embeddings(torch.tensor(tokens)) #Her kelimenin ID'sine karşılık anlam vektörü

In [ ]:
meanings.shape

In [ ]:
pos_meanings = pos_embeddings(torch.tensor([i for i in range(tokens.shape[0])]))

In [ ]:
pos_meanings.shape

In [ ]:
pos_meanings

In [ ]:
meanings_in_context = meanings + pos_meanings

In [ ]:
meanings_in_context.shape

In [ ]:
sentences = [
  {
    "words": meanings_in_context.detach().numpy(),
    "labels": tokenizer.tokenize(sentence),
    "color": "red",
  },
]
plot_dots(sentences, "Basic Positional Embeding")

kelime embedding’i ‘ne’ olduğunu, positional embedding ‘nerede’ olduğunu söyler; ikisini toplayınca model hem anlamı hem sırayı aynı vektörde görür

##Sinusoidal Positional Encoding



*   GPT-2 ve Bertte kullanılan yöntem budur
*   Eğitilmesine gerek olmadan daha önce görmediği uzunluktaki cümlelerle de genelleştirilebilir
*   Her boyuta "frekans" veriyoruz ve böylelikle bazı boyutlar yavaş değişirken bazıları hızlı değişiyor





In [ ]:
p_embeddings = torch.zeros(context_length, 4, device='cpu') #boş tensor

In [ ]:
p_embeddings

In [ ]:
import math

In [ ]:
def get_position_encoding(context_length, embedding_dim, base=10000, device="cpu"): #base -> Frekans ölçekleme tabanı - sinüs/kosinüs dalgalarının frekansını ayarlar
  p_embeddings = torch.zeros(context_length, embedding_dim, device=device)
  for pos in range(context_length):
    for i in range(embedding_dim // 2):
      p_embeddings[pos, 2 * i] = math.sin(pos / (base ** (2 * i / embedding_dim)))
      if i + 1 < embedding_dim:
        p_embeddings[pos, 2 * i + 1] = math.cos(pos / (base ** (2 * i / embedding_dim)))

  return p_embeddings.unsqueeze(0) # unsqueeze ?

In [ ]:
# pos=0: [ sin(0/f0), cos(0/f0), sin(0/f1), cos(0/f1) ] = [0, 1, 0, 1]
# pos=1: [ sin(1/f0), cos(1/f0), sin(1/f1), cos(1/f1) ]
# pos=2: [ sin(2/f0), cos(2/f0), sin(2/f1), cos(2/f1) ]

In [ ]:
pos_embeddings = get_position_encoding(context_length, 4)

In [ ]:
pos_embeddings.shape

In [ ]:
pos_embeddings

In [ ]:
pos_embeddings = get_position_encoding(20, 4)

In [ ]:
meanings_in_sentence = meanings + pos_embeddings

In [ ]:
meanings_in_sentence

In [ ]:
sentences = [
  {
    "words": meanings_in_sentence[0].detach().numpy(),
    "labels": tokenizer.tokenize(sentence),
    "color": "red",
  },
  {
    "words": meanings.detach().numpy(),
    "labels": tokenizer.tokenize(sentence),
    "color": "blue",
  },
]
plot_dots(sentences, "Sentence Context Space")

*   Böylelikle kelimelerin cümle içindeki anlamlarını bulmuş olduk
*   Kelimeler sözlükte yakın olmasalar bile aynı cümlede olmalarıyla birlikte anlamsal olarak yakın olabilirler

##Rotary Position Encoding (RoPE)

*   DeepSeek en son bunu kullandı
*   Hem kelimelerin cümle içindeki sırası hem de birbirlerine olan uzaklığını hesaplayan bir yöntem
*   Vektörlerin açısını değiştireceğiz

In [ ]:
freqs_indices = torch.arange(0, 20, dtype=torch.float32)

In [ ]:
freqs_indices

In [ ]:
freqs = 1.0 / (10000 ** (freqs_indices / 20)) #düşük i yavaş değişim - yüksek i hızlı değişim

*   düşük i → yavaş değişen dalga, uzak ilişkiler
*   yüksek i → hızlı değişen dalga, yakın ilişkiler

In [ ]:
freqs.unsqueeze(1) #unsqueezesiz dene

Cümlenin başından sonuna doğru önemi azalıyor

In [ ]:
def get_rotary_position_encoding(input: torch.Tensor, base=10000, device="cpu"):
  context_length, dimension = input.shape

  assert dimension % 2 == 0, "boyutlar eşit olmalıdır"

  half_dimension = dimension // 2

  freqs_indices = torch.arange(0, half_dimension, device=device, dtype=torch.float32)
  freqs = 1.0 / (base ** (freqs_indices / dimension))

  positions = torch.arange(0, context_length, device=device, dtype=torch.float32).unsqueeze(1)

  angles = positions * freqs # Pozisyonlar ile frekansları çarparak her pozisyon ve her boyut için açı hesaplanır

  sin_angles = torch.sin(angles)
  cos_angles = torch.cos(angles)

  input_even = input[:, :dimension // 2] # [0, 2, 4, ..]
  input_odd = input[:, dimension // 2:] # [1, 3, 5, ..]

  # 2D rotasyon formülü uygulanır:
  # [x', y'] = [x*cos - y*sin, x*sin + y*cos]
  input_even_rotated = input_even * cos_angles - input_odd * sin_angles
  input_odd_rotated = input_even * sin_angles + input_odd * cos_angles

  input_rotated = torch.empty_like(input) # Rotasyon sonrası değerleri orijinal tensör boyutunda yeni tensöre yerleştirir

  # İlk yarıya döndürülmüş even, ikinci yarıya döndürülmüş odd bileşenler yazılır
  input_rotated[:, :dimension // 2] = input_even_rotated
  input_rotated[:, dimension // 2:] = input_odd_rotated

  return input_rotated

In [ ]:
torch.manual_seed(42)

In [ ]:
random_input = torch.rand(context_length, 4)

In [ ]:
random_input

In [ ]:
pos_rotary_encodings = get_rotary_position_encoding(random_input)

In [ ]:
pos_rotary_encodings

In [ ]:
meanings_with_pos_encodings = get_rotary_position_encoding(meanings)

In [ ]:
meanings_with_pos_encodings

In [ ]:
sentences = [
  {
    "words": meanings_with_pos_encodings.detach().numpy(),
    "labels": tokenizer.tokenize(sentence),
    "color": "red",
  },
  {
    "words": meanings.detach().numpy(),
    "labels": tokenizer.tokenize(sentence),
    "color": "blue",
  },
]
plot_dots(sentences, "Sentence Context Space V2 rotary")